In [37]:
import numpy as np

kernel = np.array([
        [0,0,1,0,0],
        [0,0,2,0,0],
        [1,2,3,2,1],
        [0,0,2,0,0],
        [0,0,1,0,0],
    ])

target = np.array([
    [189] * 5 + [33] * 3 + [189] * 4,
    [189] * 3 + [33] * 3 + [189,33,44] + [189] * 3,
    [189] * 5 + [33] * 4 + [189] * 3,
    [189] * 5 + [33,189,33,33] + [189] * 3,
    [189] * 3 + [33,33,189,189,33,33,33] + [189] * 2,
    [189,134] + [33] * 2 + [189] * 4 + [33] * 2 + [189] * 2,
    [189,144] + [33] * 2 + [189] * 4 + [33] + [189] * 3,
    [189,142] + [33] * 2 + [189] * 4 + [33] * 3 + [189],
    [189,100,142,33] + [189] * 4 + [33] * 3 + [189],
    [189] + [142] * 2 + [189] * 6 + [33] + [189] * 2,
    [189,59,142,33] + [189] * 4 + [33] + [189] * 3,
    [189] * 2 + [33] * 2 + [189] * 8,
]).reshape((144))

coefficients = np.zeros((144,144))

In [38]:
for x in range(12):
    for y in range(12):
        for dx in range(-2, 3, 1):
            for dy in range(-2, 3, 1):
                xx = x + dx
                yy = y + dy
                if xx < 0 or xx >= 12 or yy < 0 or yy >= 12:
                    continue
                if (tmp := kernel[dx + 2][dy + 2]) > 0:
                    coefficients[x * 12 + y][xx * 12 + yy] = tmp

In [39]:
for i in range(144):
    assert(coefficients[i][i] != 0)

In [40]:
from Crypto.Util.number import *

def gauss(c, d, modulus = 256):
    size = c.shape[0]
    assert(c.shape[0] == c.shape[1])
    for i in range(size - 1):
        for j in range(i + 1, size):
            while c[j][i] != 0:
                t = c[j][i] // c[i][i]

                c[j] = (c[j] - t * c[i]) % modulus
                d[j] = (d[j] - t * d[i]) % modulus

                if c[j][i] != 0:
                    c[[i, j]] = c[[j, i]]
                    d[[i, j]] = d[[j, i]]
    
    for i in range(size - 1, -1 ,-1):
        d[i] -= sum([(c[i][j] * d[j]) % modulus for j in range(i + 1, size)])
        d[i] %= modulus

        if d[i] % c[i][i] == 0:
            d[i] //= c[i][i]

        else:
            d[i] *= inverse(c[i][i], modulus)
            d[i] %= modulus
            
    return d

def verify(table, ans, expected, modulus = 256):
    ress = []
    for line in table:
        res = 0
        for x, c in zip(ans, line):
            res += c * x
        res %= modulus
        ress.append(res)
    return ress == expected.T

In [41]:
table = np.array([
    [2,1,1],
    [6,2,1],
    [-2,2,1]])
b = np.array([1,255,7])

ans = gauss(table.copy(), b.copy())
verify(table, ans, b)

array([ True,  True,  True])

In [42]:
ans = gauss(coefficients.copy().astype('int64'), target.copy())
verify(coefficients, ans, target)

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,

In [43]:
import win32api
import win32con
import time

def click_cur(pos, times):
    win32api.SetCursorPos(pos)
    for _ in range(times):
        win32api.mouse_event(win32con.MOUSEEVENTF_LEFTDOWN|win32con.MOUSEEVENTF_LEFTUP,0,0,0,0)

def get_cur():
    print('[+] waiting...',end='')
    for i in range(5):
        time.sleep(1)
        print(5 - i,end='...')
    print()
    return win32api.GetCursorPos()

def init():
    print('[+] please place your mouse the block at (0,0) in 5s')
    origin = get_cur()
    print(f'[!] (0,0) at {origin}')
    print('[+] please place your mouse the block at (0,1) in 5s')
    now = get_cur()
    print(f'[!] (0,1) at {now}')
    step = now[0] - origin[0]
    return origin, step

In [44]:
origin, step = init()
ans = ans.reshape((12,12))
for i in range(12):
    for j in range(12):
        click_cur((origin[0] + step * j, origin[1] + step * i), ans[i][j])
        time.sleep(0.1)

In [45]:
from scipy import signal
signal.convolve2d(ans.reshape((12,12)), kernel, 'same') % 256 - target.reshape((12,12))

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [46]:
ans.reshape((12,12))

array([[127,  98, 128, 120, 232, 117, 161,  82,  41, 141, 137, 137],
       [ 54, 109,  93,   9,  44,  57,  86, 153, 197,  16, 123, 252],
       [144,  71, 104, 156, 178, 116, 243,  99, 208, 183, 215, 139],
       [198, 245, 136,  38,  50, 254, 254, 109, 200, 240, 175, 155],
       [160,  54,  51,  36, 136, 153, 207, 233, 112, 207,   7,   0],
       [ 99, 181, 100,  91, 185, 225,  25,  47,  29, 219, 155, 140],
       [ 92, 182,   8, 210, 133,   5,  51, 229, 241, 144,  47,   7],
       [  8, 177, 187,  46,  81, 178, 122, 255, 120, 183, 230,  95],
       [191, 210, 167, 160, 169, 200, 229, 236, 104, 238,   8, 228],
       [239,   2,  23, 221, 192, 105, 219, 162,  30,  11, 194, 187],
       [235, 102, 161,  31, 191, 130, 184, 130,  43, 147, 126, 216],
       [121, 126, 145,  97, 222, 135, 132, 150, 206,  42, 219,  38]])